In [1]:
from functions_scraping import select_pages, crawl_charts, create_file

In [2]:
import os
import pandas as pd

In [6]:
home = os.environ['HOME']
path = os.path.join(home, 'top100-data')

# create data folder in home directory if it does not exists
if not os.path.exists(path):
    os.makedirs(path)

In [9]:
# scrapring charts for year 2019
years = list(range(2019, 2020))

for year in years:
    # select pages to crawl
    start = str(year-1) + '-12-31'
    end = str(year) + '-12-31'
    pages = select_pages(start, end)
    
    # web scraping of https://www.offiziellecharts.de for chosen dates
    charts = crawl_charts(pages)
    
    # create file for year
    create_file(charts, os.path.join(path, str(year)+'_top_100'))

Request:53; Frequency: 0.08454849055944683 requests/s


In [29]:
# create empty dataframe
alltime = pd.DataFrame()

# combine pickle files of all years into one dataframe 
for filename in os.listdir(path):
    if filename.endswith("_top_100"):
        df = pd.read_pickle(filename)
        alltime = alltime.append(df, ignore_index=True)

In [32]:
# save dataframe to pickle
alltime.to_pickle(os.path.join(path, 'alltime_top_100'))